# WHT ISIS wavelength calibration

The following code compliments the basic WHT ISIS long slit data reduction scripts. This code can either be implemented separately, or can be added to the basic reduction scripts to create a full pipeline.

In [1]:
import os
import numpy as np

from astropy.io import fits
import astropy.units as u

from mpdaf.obj import Spectrum, WaveCoord

import matplotlib.pyplot as plt


### Define dispersion functions for red and blue arms
def solution_b(x, m, c):
    return m*x + c

def solution_r(x, m, c, n, d, o, e):
    return m*(x**c) + n*(x**d) + o*(x**e)

def gauss(x, *p):
    A, mu, sigma = p
    return A*np.exp(-(x-mu)**2/(2.*sigma**2))

In [ ]:
### Change to the directory with data

os.chdir("$HOME/WHTRUN/20170626/")

## Blue arm

In [ ]:
### Select target and the relevant arc

### select target
target = "target"

### select the relevant arc
blue_arc = fits.getdata("../ARCS/arc_blue.fit")

print("Arc loaded.")

In [ ]:
### normalise the spectrum of arc
mean_blue_arc = np.median(blue_arc[:, :], axis=0)

# print(len(mean_blue_arc))

## normalise the arc lines
maxval = np.max(mean_blue_arc)
blue_arc_normed = mean_blue_arc/maxval


### Show the normalised arc spectrum for visual inspection
%matplotlib notebook

fig = plt.figure(figsize=(9,5))
plt.plot(blue_arc_normed)

plt.grid(alpha=0.4)
plt.show()

In [ ]:
### We now identify key features in the arc and then match them to the features shown in the WHT arcs PDF
### The arc files can be found here: http://www.ing.iac.es/Astronomy/instruments/isis/calibration_info.html

### Given below are some starting pixel values for 2x2 and 1x1 binning for CuNe + CuAr arcs.
### The pixel values and features change depending on the choice of arc, central wavelength and binning.


### For 2x2 binning
# pixels_blue = [338.6, 504.1, 818.5, 837.5, 853.5, 887.5, 927.6, 
#                 1197.3, 1243.6, 1268.1, 1335.2, 1415.5, 1481.6, 1545.9, 1586.5, 1899.32]

# wavelengths_blue = [3273.96, 3530.39, 4072., 4103.91, 4131.72, 4190.71, 4259.36, 
#                 4726.87, 4806.02, 4847.81, 4965.08, 5105.54, 5218.20, 5330.78, 5400.56, 5944.83]


### For 1x1 binning:

# Lines for the blue arc
pixels_blue = [1176.38, 1492.16, 2104.31, 2140.17, 2171.93, 2240.57, 2319.45, 
                2861.92, 2952.51, 3000.46, 3135.45, 3297.08, 3428.51, 3558.17, 3638.10]

wavelengths_blue = [3273.96, 3530.39,  4072., 4103.91, 4131.72, 4190.71, 4259.36, 
                4726.87, 4806.02, 4847.81, 4965.08, 5105.54, 5218.20, 5330.78, 5400.56 ]


# Create x axis
### For 2x2 binning
# xaxis = np.linspace(0,2049, 2049)

### For 1x1 binning
xaxis = np.linspace(1,4096, 4096)


line_pix = []

%matplotlib notebook

fig = plt.figure(figsize=(9,5))
plt.grid(alpha=0.4)
plt.plot(blue_arc_normed)

### Automatically fit Gaussians at the positions of identified arc features
for pixel in pixels_blue:
    # create gaussian to fit to the arc line
    p0 = [0.4, pixel, 3.0]
    coeffs, covar = optimize.curve_fit(gauss, xaxis[int(pixel)-12:int(pixel)+12], blue_arc_normed[int(pixel)-12:int(pixel)+12], p0=p0, 
                                       bounds=([0.05, pixel-12, 1.0], [1.0, pixel+12, 6.0]))
    line_pix.append(coeffs[1])
    plt.plot(np.linspace(pixel-12, pixel+12, 10), gauss(np.linspace(pixel-12, pixel+12, 10), *coeffs))
    
    
sol_blue = []
for i in range(len(wavelengths_blue)):
    sol_blue.append(wavelengths_blue[i]-line_pix[i])

# %matplotlib notebook
# plt.plot(line_pix, wavelengths_blue)

In [ ]:
# Lets try and find a wavelength calibration solution
# Fit Blue wavelength solutions

# initial guess
p0 = [0.87, 2200]

fit_blue, cov_blue = optimize.curve_fit(solution_b, line_pix, wavelengths_blue, p0=p0)

print(fit_blue)

waveblue = solution_b(xaxis, *fit_blue)

%matplotlib notebook

plt.plot(line_pix, wavelengths_blue, 'ko')
plt.plot(xaxis, waveblue)

plt.show()

In [ ]:
### Calculate CRPIX and CRDELT values based on new wavelength range and then update the header of the SCI file

crval1 = waveblue[0]
cdelt1 = (waveblue[-1]-waveblue[0])/len(waveblue)
cunit1 = 'Angstrom'

print(crval1, cdelt1, cunit1)

## Apply the wavelength calibration to the 2D fits spectrum by updating the header

scidata, scihead = fits.getdata("./processed/%s_blue.fits" %target, header=True)

scihead['CRVAL1'] = crval1
scihead['CRVAL2'] = 0.0
scihead['CRPIX2'] = 525.0
scihead['CDELT1'] = cdelt1
scihead['CUNIT1'] = cunit1
scihead['CUNIT2'] = 'Arcsec'
scihead['CTYPE1'] = 'LINEAR'
scihead['CTYPE2'] = 'LINEAR'
scihead['CD1_1'] = cdelt1
scihead['CD2_2'] = 0.2
scihead['PV2_1'] = 1.0

fits.writeto("./processed/SCI/%s_blue_WC.fits" %target, scidata, header=scihead, overwrite=True)

print("Blue wavelength calibration complete.")

## Red arm

In [ ]:
### Select target and the relevant arc

# target = "target"
red_arc = fits.getdata('../ARCS/arc_red.fit')

print("Red arc loaded.")

In [ ]:
### normalise the spectrum
mean_red_arc = np.median(red_arc[:, :], axis=0)

## normalise the arc lines
maxval = np.max(mean_red_arc)
red_arc_normed = mean_red_arc/maxval

%matplotlib notebook

fig = plt.figure(figsize=(9,5))
plt.plot(red_arc_normed)

plt.grid(alpha=0.4)
plt.show()

In [ ]:
# Lines for the red arc

# For 1x1 binning
pixels_red = [1340.59, 1463.73, 1501.84, 1570.44, 1607.38, 1645.5,
         1702.46, 1798.27, 1936.21, 1993.06, 2110.99,2168.8,
             2496.6, 2671.29, 2812.62, 3141.35, 3196.93, 3435.14]
wavelengths_red = [5852.49, 6074.34, 6143.06, 6266.50, 6334.43, 6402.25,
              6506.53, 6678.2, 6929.47, 7032.41, 7245.17, 7383.98,
                  7948.17, 8264.52, 8521.44, 9122.97, 9224.5, 9657.78]


### For 2x2 binning
# pixels_red = [544.8, 605.2, 624.5, 659.3, 678.4, 696.7, 725.5, 772.7,
#               842.0, 870.9, 929.6, 967.2, 
#               1122.6, 1209.5, 1320.5, 1444.3, 1472.9, 1591.7]

# wavelengths_red = [5852.49, 6074.34, 6143.06, 6266.50, 6334.43, 6402.25, 6506.53, 6678.2,
#                    6929.47, 7032.41, 7245.17, 7383.98, 
#                    7948.17, 8264.52, 8521.44, 9122.97, 9224.5, 9657.78]


# Create x axis
xaxis = np.linspace(1,4096, 4096)

%matplotlib notebook

fig = plt.figure(figsize=(9,5))
plt.grid(alpha=0.4)
plt.plot(red_arc_normed)

line_pix = []

for pixel in pixels_red:
    # create gaussian to fit to the arc line
    p0 = [0.5, pixel, 4.0]
    coeffs, covar = optimize.curve_fit(gauss, xaxis[int(pixel)-20:int(pixel)+20], red_arc_normed[int(pixel)-20:int(pixel)+20], p0=p0, 
                                       bounds=([0.1, pixel-10, 1.0], [1.0, pixel+10, 7.0]))
    line_pix.append(coeffs[1])
    plt.plot(np.linspace(pixel-10, pixel+10, 10), gauss(np.linspace(pixel-10, pixel+10, 10), *coeffs))
    
    
sol_red = []
for i in range(len(wavelengths_red)):
    sol_red.append(wavelengths_red[i]-line_pix[i])

plt.grid(alpha=0.4)
plt.show()

# %matplotlib notebook
# plt.plot(line_pix, wavelengths_red)

In [ ]:
# Lets try and find a wavelength calibration solution
# Fit Red wavelength solutions

# initial guess
p0 = [1.8, 3400]

fit_red, cov_red = optimize.curve_fit(solution_b, line_pix, wavelengths_red, p0=p0)

print(fit_red)

wavered = solution_b(xaxis, *fit_red)

%matplotlib notebook

plt.plot(line_pix, wavelengths_red, 'ko')
plt.plot(xaxis, wavered)

plt.show()

In [ ]:
### Calculate CRPIX and CRDELT values based on new wavelength range and then update the header of the SCI file
# print(len(wavered))
# print(wavered[0], max(wavered))

crval1 = wavered[0]
cdelt1 = (wavered[-1]-wavered[0])/len(wavered)
cunit1 = 'Angstrom'

print(crval1, cdelt1, cunit1)

## Update sci file

scidata, scihead = fits.getdata("./processed/%s_red.fits" %target, header=True)

scihead['CRVAL1'] = crval1
scihead['CRVAL2'] = 1.0
scihead['CDELT1'] = cdelt1
scihead['CRPIX2'] = 212.0
scihead['CUNIT1'] = cunit1
scihead['CUNIT2'] = 'Arcsec'
scihead['CTYPE1'] = 'LINEAR'
scihead['CTYPE2'] = 'LINEAR'
scihead['CD1_1'] = cdelt1
scihead['CD2_2'] = 0.44
scihead['PV2_1'] = 1.0

fits.writeto("./processed/SCI/%s_red_WC.fits" %target, scidata, header=scihead, overwrite=True)

print("Red wavelength calibration complete.")

## Extraction of 1D spectrum from 2D

Import the wavelength calibrated 2D spectrum and extract a 1D spectrum using an aperture. We will save the output spectrum as an MPDAF Spectrum object

In [ ]:
## BLUE ARM
## Read in the 2D spectrum

spec2d, spechead = fits.getdata("./processed/SCI/target_blue_WC.fits", header=True)

## Read in the wavelength axis values
cdelt = spechead['CDELT1']
crval = spechead['CRVAL1']

wavelength_1d = WaveCoord(cdelt=cdelt, crval=crval, cunit=u.Angstrom)

### Select the central pixel and aperture size (depending on the 2D spectrum)
central_pix = 495
aperture = 10

spectrum_1d = np.sum(spec2d[central_pix-aperture:central_pix+aperture,:], axis=0)
# print(np.shape(spectrum_1d))

spectrum_mpdaf_b = Spectrum(wave=wavelength_1d, data=spectrum_1d, data_header=spechead)

%matplotlib notebook
fig = plt.figure(figsize=(10,3))
plt.grid(alpha=0.4)

spectrum_mpdaf_b.rebin(2).plot()

# plt.xlim(5500,9600)
# plt.ylim(-30,100)

plt.show()

In [ ]:
### Save only the high throughput wavelengths to spectrum 
### (this step is not required but recommended for a clean spectrum)

scispec_mpdaf_b = spectrum_mpdaf_b.subspec(lmin=3500, lmax=5700, unit=u.Angstrom)
scispec_mpdaf_b.write("./processed/SCI/target_blue_1D.fits")

print("1D spectrum extracted")

In [ ]:
## RED ARM
## Read in the 2D spectrum

spec2d, spechead = fits.getdata("./processed/SCI/target_red_WC.fits", header=True)

## Read in the wavelength axis values
cdelt = spechead['CDELT1']
crval = spechead['CRVAL1']

wavelength_1d = WaveCoord(cdelt=cdelt, crval=crval, cunit=u.Angstrom)

central_pix = 483

aperture = 5

spectrum_1d = np.sum(spec2d[central_pix-aperture:central_pix+aperture,:], axis=0)
# print(np.shape(spectrum_1d))

spectrum_mpdaf_r = Spectrum(wave=wavelength_1d, data=spectrum_1d, data_header=spechead)

### show 1D spectrum

%matplotlib notebook

fig = plt.figure(figsize=(10,5))
plt.grid(alpha=0.4)

spectrum_mpdaf_r.plot()

plt.show()

In [ ]:
### write the spectrum

scispec_mpdaf_r = spectrum_mpdaf_r.subspec(lmin=5400, lmax=10000, unit=u.Angstrom)
scispec_mpdaf_r.write("./processed/SCI/target_red_1D.fits")

print("1D spectrum extracted")